In [1]:
import pandas as pd
import pandasql as ps

Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах). В отношении Ships записаны название корабля, имя его класса и год спуска на воду. В отношение Battles включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.

![](scheme1.png)

In [2]:
classes = pd.read_csv('classes.csv', index_col='class')
ships = pd.read_csv('ships.csv', index_col='name')
outcomes = pd.read_csv('outcomes.csv', index_col=['ship', 'battle'])
battles = pd.read_csv('battles.csv', index_col='name')

Задача 1. Найдите страны, имевшие когда-либо классы обычных боевых кораблей ('bb') и имевшие когда-либо классы крейсеров ('bc').

![](scheme1.png)

In [4]:
query = '''
    select country
    from classes
    group by country
    having count(distinct type) = 2
'''
ps.sqldf(query)

,country
0,Gt.Britain
1,Japan


In [12]:
# Использовать groupby и nunique
# Решить одной командой без использования merge и без дополнительных переменных
# Разрешается отдельно не выделять True страны

classes.groupby('country').nunique().loc[classes.groupby('country').nunique()['type'] == 2, []].reset_index()

,country
0,Gt.Britain
1,Japan


Задача 2.  Для каждого класса определите год, когда был спущен на воду первый корабль этого из класса.
![](scheme1.png)

In [19]:
query = '''
    select a.class, c
    from(
    select
        class
    from classes) a
    left join(
    select
        class, min(launched) as c
    from ships
    group by class) b
    on a.class = b.class
'''
ps.sqldf(query)

,class,c
0,Bismarck,NaN
1,Iowa,1943.0
2,Kongo,1913.0
3,North Carolina,1941.0
4,Renown,1916.0
5,Revenge,1916.0
6,Tennessee,1920.0
7,Yamato,1941.0


In [22]:
# Применить merge, groupby, min
# Решить одной командой без введения дополнительных переменных

pd.merge(classes.loc[classes.index, []], ships.groupby('class').min(), how='left', left_on='class', right_index=True).reset_index()

,class,launched
0,Bismarck,NaN
1,Iowa,1943.0
2,Kongo,1913.0
3,North Carolina,1941.0
4,Renown,1916.0
5,Revenge,1916.0
6,Tennessee,1920.0
7,Yamato,1941.0


Таблица company содержит идентификатор и название компании, осуществляющей перевозку пассажиров. Таблица trip содержит информацию о рейсах: номер рейса, идентификатор компании, тип самолета, город отправления, город прибытия, время отправления и время прибытия. Таблица passenger содержит идентификатор и имя пассажира. Таблица pass_in_trip содержит информацию о полетах: номер рейса, дата вылета (день), идентификатор пассажира и место, на котором он сидел во время полета.


![](scheme2.png)

In [97]:
# Загрузка таблиц

trip = pd.read_csv('trip.csv', index_col='trip_no',
    parse_dates=['time_out', 'time_in'], infer_datetime_format=True)
pass_in_trip = pd.read_csv('pass_in_trip.csv',
    infer_datetime_format=True, parse_dates=['date'])
company = pd.read_csv('company.csv', index_col='id_comp')
passenger = pd.read_csv('passenger.csv', index_col='id_psg')

Задача 3. Определить имена пассажиров, когда-либо летевших на одном и том же месте более одного раза. 

![](scheme2.png)

In [24]:
query = '''
select name
from passenger
join (
    select *
    from pass_in_trip
    group by id_psg, place
    having count(place) > 1
) b
on passenger.id_psg = b.id_psg
'''
ps.sqldf(query)

,name
0,Bruce Willis
1,Nikole Kidman
2,Mullah Omar


In [35]:
# Решить одной командой используя один merge и два groupby, функции count и sum
# Запрещается введение дополнительных переменных

pd.merge(pass_in_trip.groupby(['place', 'id_psg']).count().loc[(pass_in_trip.groupby(['place', 'id_psg']).count() >= 2)['trip_no'], []],
         passenger, how='left', left_on='id_psg', right_index=True).reset_index(['place', 'id_psg']).drop(['place', 'id_psg'], axis=1)

,name
0,Bruce Willis
1,Mullah Omar
2,Nikole Kidman


Задача 4.

Для каждой авиакомпании определить:

* количество выполненных перелетов;
* число использованных типов самолетов;
* количество перевезенных различных пассажиров;
* общее число перевезенных компанией пассажиров. 

![](scheme2.png)

In [78]:
query = '''
select count(id_psg), count(distinct id_psg), count(distinct plane), count(distinct a.trip_no)
from 
trip a left join pass_in_trip b
on a.trip_no = b.trip_no
group by id_comp
'''
ps.sqldf(query)

,count(id_psg),count(distinct id_psg),count(distinct plane),count(distinct a.trip_no)
0,9,6,2,6
1,2,2,1,2
2,4,4,1,2
3,1,1,1,2
4,16,6,1,10


In [73]:
# Решить одной командой используя nunique
# Запрещается введение дополнительных переменных

pd.merge(pd.merge(pass_in_trip, trip.loc[trip.index,['id_comp']],
                  left_on='trip_no', right_index=True).groupby('id_comp').count(),
         pd.merge(pass_in_trip.loc[pass_in_trip.index, ['trip_no', 'id_psg']], trip.loc[trip.index,['id_comp', 'plane']],
                  how='outer', left_on='trip_no', right_index=True).groupby('id_comp').nunique().drop('id_comp',axis=1),
         left_on='id_comp', right_index=True).reset_index().drop(['date', 'place', 'id_comp', 'trip_no_x'], axis=1)

,id_psg_x,trip_no_y,id_psg_y,plane
0,9,6,6,2
1,2,2,2,1
2,4,2,4,1
3,1,2,1,1
4,16,10,6,1


Задача 5. Найти количество маршрутов, которые обслуживаются наибольшим числом рейсов.
Замечания.
* A - B и B - A считать разными маршрутами.
* Использовать только таблицу Trip 
![](scheme2.png)

In [42]:
query = '''
select count(*) from (
        select count(distinct a.trip_no) as cnt
        from trip a
        group by a.town_from, a.town_to
    ) c
    where cnt = (
        select max(cnt) from 
        (
            select count(distinct a.trip_no) as cnt
            from trip a
            group by a.town_from, a.town_to
        )
)'''
ps.sqldf(query)

,count(*)
0,4


In [96]:
# Решить одной командой
# Не допускается ипользование merge и дополнительных переменных

(trip.groupby(['town_to', 'town_from']).count()['id_comp'] == trip.groupby(['town_to', 'town_from']).count()['id_comp'].max()).sum()

4

Задача 6. Определить номера пассажиров, которые больше других времени провели в полетах.
Вывести имена и общее время в минутах, проведенное в полетах

![](scheme2.png)

In [95]:
# Решить одной командой без использования дополнительных переменных
# Здесь расматриваются типы времени: изначально timestamp, и после взятия разности -- timediff.
# Обращение к методам времени осуществляется через поле series.dt
# Использовать pd.DataFrame для создания промежуточного массива времен, groupby, sum и idxmax

# Жень, я старался сделать понятнее, но это выше моих сил, делать такие запросы в 1 команду - это жесть несусветная

pd.merge(pd.merge(pass_in_trip, pd.DataFrame((trip['time_in'] - trip['time_out']).dt.seconds / 60, columns = ['time_in_travel']),
         left_on='trip_no', right_index=True).groupby('id_psg').sum().loc[
    pd.merge(pass_in_trip, pd.DataFrame((trip['time_in'] - trip['time_out']).dt.seconds / 60, columns = ['time_in_travel']),
         left_on='trip_no', right_index=True).groupby('id_psg').sum().index ==
    pd.merge(pass_in_trip, pd.DataFrame((trip['time_in'] - trip['time_out']).dt.seconds / 60, columns = ['time_in_travel']),
         left_on='trip_no', right_index=True).groupby('id_psg').sum()['time_in_travel'].idxmax()], passenger,
         how='left', left_on='id_psg', right_index=True).reset_index().drop(['trip_no', 'id_psg'], axis=1)

,time_in_travel,name
0,2100.0,Michael Caine
